In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import warnings

#hourly_smp data는 시간형식이 일반적인 형태와는 다르게 되어있음
#이를 위해 따로 date_time_split_2를 제작
from processing import date_time_split_1
from processing import date_time_split_2

warnings.filterwarnings(action = 'ignore')
pd.set_option('display.max_columns', 500)
%matplotlib inline

# 1. Data Load

In [2]:
target_original = pd.read_csv('data/target_v1.csv')
weather_original = pd.read_csv('data/weather_v1.csv')
hourly_smp_original = pd.read_csv('data/hourly_smp_v1.csv')

target = target_original.copy()
weather = weather_original.copy()
hourly_smp = hourly_smp_original.copy()

In [3]:
oil_price_cl_original = pd.read_csv('result/oil_price_cl.csv')
oil_price_du_original = pd.read_csv('result/oil_price_du.csv')
oil_price_brt_original = pd.read_csv('result/oil_price_brt.csv')

oil_price_cl = oil_price_cl_original.copy()
oil_price_du = oil_price_du_original.copy()
oil_price_brt = oil_price_brt_original.copy()

## area column이 여러개 존재
### 가장 null값이 적은 area를 선택

In [4]:
area_list = weather['area'].unique()
for ls in area_list :
    null_temp  = weather[weather['area'] == ls]['temp'].isnull().sum()
    print('{} total shape : {}'.format(ls, len(weather[weather['area'] == ls]['temp'])))
    print('{} null shape : {}'.format(ls, null_temp))

184 total shape : 17518
184 null shape : 2
185 total shape : 17512
185 null shape : 20
188 total shape : 17518
188 null shape : 7
189 total shape : 17516
189 null shape : 11
328 total shape : 17488
328 null shape : 72
329 total shape : 17514
329 null shape : 5
330 total shape : 17432
330 null shape : 4
724 total shape : 17423
724 null shape : 7
725 total shape : 17469
725 null shape : 11
726 total shape : 17392
726 null shape : 108
727 total shape : 17517
727 null shape : 4
751 total shape : 17438
751 null shape : 12
752 total shape : 17518
752 null shape : 21
753 total shape : 17287
753 null shape : 15
779 total shape : 17518
779 null shape : 3
780 total shape : 17244
780 null shape : 53
781 total shape : 17399
781 null shape : 22
782 total shape : 17515
782 null shape : 3
792 total shape : 17514
792 null shape : 9
793 total shape : 17467
793 null shape : 132
855 total shape : 17448
855 null shape : 503
861 total shape : 17510
861 null shape : 3
862 total shape : 17005
862 null shape 

**daily smp data 수집 기간**  
**2018년 2월 1일 ~ 2020년 1월 31일**

**데이터 분석 결과 온도를 제외하고는 날씨 데이터에서는 유의미한 변수가 없는것으로 판단**  
**weather data 에서는 temp column만을 추출**

**oil data 수집기간**  
**2006년 4월 18일 ~ 2020년 5월 11일(주말 제외)**

# 2. EDA

## 형식이 유사한 weather와 hourly_smp data 먼저 통합

### weather dataset을 비롯한 모든 데이터의 필요없는 column을 drop하고 변수명을 변환

In [5]:
weather = weather[weather['area'] == 884]
weather = weather.iloc[:, 1:3]
weather.reset_index(drop = True, inplace = True)
weather.columns = ['date', 'temp']
weather = date_time_split_1(weather)

In [6]:
hourly_smp.columns = ['dates', 'smp']
hourly_smp = date_time_split_2(hourly_smp)

In [7]:
weather.head()

,temp,year,month,day,weekday,hour
0,3.3,2018,2,1,3,1
1,2.2,2018,2,1,3,2
2,3.8,2018,2,1,3,3
3,0.7,2018,2,1,3,4
4,2.4,2018,2,1,3,5


In [8]:
weather_smp = pd.merge(weather, hourly_smp, on = ['year', 'month', 'day', 'weekday', 'hour'], how = 'outer')
weather_smp.head()

,temp,year,month,day,weekday,hour,smp
0,3.3,2018,2,1,3,1,116.84
1,2.2,2018,2,1,3,2,116.84
2,3.8,2018,2,1,3,3,116.84
3,0.7,2018,2,1,3,4,116.84
4,2.4,2018,2,1,3,5,116.84


## 가장 형식이 차이나는 oil data를 이후 처리

In [9]:
#oil_price에서 필요한 column(date, price)만을 추출하고 날짜 데이터를 풀어줌
oil_price_cl = oil_price_cl.iloc[:, 1:3]
oil_price_du = oil_price_du.iloc[:, 1:3]
oil_price_brt = oil_price_brt.iloc[:, 1:3]

oil_price_cl.columns = ['date', 'cl_price']
oil_price_du.columns = ['date', 'du_price']
oil_price_brt.columns = ['date', 'brt_price']

### 각 table의 날짜 데이터를 date format으로 변환

**2018년 2월 1일 이전의 데이터는 모두 drop**

In [10]:
# target data의 날짜에 맞춰 sampling
# oil의 가격은 3개월 이후의 smp에 영향을 미치므로 2017-11-02 이후의 데이터만을 sampling
oil_price_du = oil_price_du[oil_price_du['date'] >= '2017-11-02'] 
oil_price_cl = oil_price_cl[oil_price_cl['date'] >= '2017-11-02']
oil_price_brt = oil_price_brt[oil_price_brt['date'] >= '2017-11-02']

oil_price_du = oil_price_du[oil_price_du['date'] <= '2019-11-02']
oil_price_cl = oil_price_cl[oil_price_cl['date'] <= '2019-11-02']
oil_price_brt = oil_price_brt[oil_price_brt['date'] <= '2019-11-02']

oil_price_du.sort_values(by = ['date'], inplace = True)
oil_price_cl.sort_values(by = ['date'], inplace = True)
oil_price_brt.sort_values(by = ['date'], inplace = True)

In [11]:
# time data를 만들기 위해 각 데이터별 날짜의 최대 최솟값을 저장해둠
period = pd.date_range(start = oil_price_du.date.min(), end = oil_price_du.date.max())

oil_price_du.reset_index(drop = True, inplace = True)
oil_price_cl.reset_index(drop = True, inplace = True)
oil_price_brt.reset_index(drop = True, inplace = True)

oil_price_cl = date_time_split_1(oil_price_cl)
oil_price_du = date_time_split_1(oil_price_du)
oil_price_brt = date_time_split_1(oil_price_brt)

In [12]:
time_data = pd.DataFrame({'date' : period})
time_data = date_time_split_1(time_data)

In [13]:
oil = pd.merge(left = time_data, right = oil_price_du, how = 'outer')
oil = pd.merge(left = oil, right = oil_price_cl, how = 'outer')
oil = pd.merge(left = oil, right = oil_price_brt, how = 'outer')
oil.head()

,year,month,day,weekday,hour,du_price,cl_price,brt_price
0,2017,11,2,3,0,58.87,54.54,60.62
1,2017,11,3,4,0,60.15,55.64,62.07
2,2017,11,4,5,0,NaN,NaN,NaN
3,2017,11,5,6,0,NaN,NaN,NaN
4,2017,11,6,0,0,61.83,57.35,64.27


In [14]:
fillna = {'du_price' : oil.du_price.mean(), 'cl_price' : oil.cl_price.mean(), 'brt_price' : oil.brt_price.mean()}
oil = oil.fillna(value = fillna)

In [15]:
oil_hour = pd.concat([oil]*24, ignore_index= True)
oil_hour.sort_values(by = ['year', 'month', 'day'], inplace = True)
oil_hour.reset_index(drop=True, inplace=True)

In [16]:
oil_hour = oil_hour.iloc[:, 5:]
oil_hour.head()

,du_price,cl_price,brt_price
0,58.87,54.54,60.62
1,58.87,54.54,60.62
2,58.87,54.54,60.62
3,58.87,54.54,60.62
4,58.87,54.54,60.62


In [17]:
oil_hour.shape

(17520, 3)

### smp, supply, oil_pirce, weather를 합친 train 제작

In [30]:
train = pd.concat([weather_smp, oil_hour], axis = 1, join = 'outer')

In [31]:
time = train['year'].astype(str) + '-' + train['month'].astype(str) + '-' + train['day'].astype(str)
train['date'] = pd.to_datetime(time)

In [34]:
train = train[['date', 'year', 'month', 'day', 'weekday', 'hour', 'temp', 'du_price', 'cl_price', 'brt_price', 'smp']]

In [35]:
train.shape

(17520, 11)

In [36]:
train.head()

,date,year,month,day,weekday,hour,temp,du_price,cl_price,brt_price,smp
0,2018-02-01,2018,2,1,3,1,3.3,58.87,54.54,60.62,116.84
1,2018-02-01,2018,2,1,3,2,2.2,58.87,54.54,60.62,116.84
2,2018-02-01,2018,2,1,3,3,3.8,58.87,54.54,60.62,116.84
3,2018-02-01,2018,2,1,3,4,0.7,58.87,54.54,60.62,116.84
4,2018-02-01,2018,2,1,3,5,2.4,58.87,54.54,60.62,116.84


## train data의 feature간 상관관계 분석

In [20]:
fig, ax = plt.subplots(figsize = (10, 10))

sns.heatmap(train.corr(), annot = True, ax = ax, fmt = '.2f')

# 3. Machine learning modelling 

### 두가지 모델로 나눔
#### 1) 이전의 smp 데이터를 바탕으로 다음 smp 예측
#### 2) 기상 데이터 및 원유가격 데이터를 바탕으로 smp 예측

## 1) keras LSTM 

### create Windows

In [21]:
seq_len = 50
seq_length = seq_len+1

result = []
for index in range(train.shape[0] - seq_length) :
    result.append(train.iloc[index : index+seq_length, :])